In [ ]:
import pandas as pd
import sqlite3
import warnings
import re
from datetime import datetime
warnings.filterwarnings(action='ignore')

In [ ]:
import os
os.environ["RUST_LOG"]="connectorx=debug,connectorx_python=debug"

In [ ]:
import connectorx as cx
import urllib

In [ ]:
conn = cx.connect("../db/hr.db")

In [ ]:
conn

In [ ]:
query = "SELECT 자료생성년월, 사업장명, 사업자등록번호, 사업장업종코드명, 가입자수, 신규취득자수, 상실가입자수, 당월고지금액 FROM pension"

In [ ]:
# %%timeit
# res = cx.read_sql(conn, query, return_type="arrow")
# df = res.to_pandas(split_blocks=False, date_as_object=False)
# df.head()

In [ ]:
%%timeit
df = cx.read_sql(conn, query, partition_on="자료생성년월", partition_num=4)
# df.head()

In [ ]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

In [ ]:
# con = sqlite3.connect("../db/hr.db")
# df = pd.read_sql_query("SELECT 자료생성년월, 사업장명, 사업자등록번호, 사업장업종코드명, 가입자수, 신규취득자수, 상실가입자수, 당월고지금액  from pension", con)

In [ ]:
df['자료생성년월'].unique()

In [ ]:
import functools # not required, but helps in production
def unpack_df_columns(func):
    """
    A general use decorator to unpack a df[subset] of columns
    into a function which expects the values at those columns
    as arguments
    """
    
    @functools.wraps(func)
    def _unpack_df_columns(*args, **kwargs):
        
        # args[0] is a pandas series equal in length as the 
        # df[subset] to which the apply function is applied 
        series = args[0]

        # series.values holds the number of arguments expected
        # by func and is os length len(df[subset].columns)
        return func(*series.values)

    return _unpack_df_columns

In [ ]:
def date_change(val):
    if val == 'Jul-22':
        return '2022-07'
    elif val == 'Jun-22':
        return '2022-06'
    elif val == 'May-22':
        return '2022-05'
    elif val == 'Apr-22':
        return '2022-04'
    elif val == 'Mar-22':
        return '2022-03'
    elif val == 'Feb-22':
        return '2022-02'
    elif val == 'Jan-22':
        return '2022-01'
    
    elif val == 'Jan-21':
        return '2021-01'
    elif val == 'Feb-21':
        return '2021-02'
    elif val == 'Mar-21':
        return '2021-03'
    elif val == 'Apr-21':
        return '2021-04'
    elif val == 'May-21':
        return '2021-05'
    elif val == 'Jun-21':
        return '2021-06'
    elif val == 'Jul-21':
        return '2021-07'
    elif val == 'Aug-21':
        return '2021-08'
    elif val == 'Sep-21':
        return '2021-09'
    elif val == 'Oct-21':
        return '2021-10'
    elif val == 'Nov-21':
        return '2021-11'
    elif val == 'Dec-21':
        return '2021-12'
    
    else:
        return val

In [ ]:
def 월평균가입자수(가입자수):
    return int(가입자수/12)

In [ ]:
@unpack_df_columns
def get_평균소득월액(고지액, 인원):
    평균소득월액 = (int(고지액) / int(인원)) / 0.09
    return int(평균소득월액)

In [ ]:
# df1 = df.copy()
# df1.info()

## 경쟁사 Data Load

In [ ]:
comp_df = pd.read_excel("./company.xlsx")
comp_df

In [ ]:
# comp_df["사업장명"].tolist()

In [ ]:
df = df.loc[(df1["사업장명"].isin(comp_df["사업장명"].tolist())) & (df["사업자등록번호"].isin(comp_df["사업자등록번호"].tolist()))]

In [ ]:
df = pd.merge(df,comp_df, on=['사업장명','사업자등록번호'], how='left')

## 날짜 변환

In [ ]:
df['자료생성년월'] = df['자료생성년월'].apply(date_change)

In [ ]:
df['자료생성년월'].unique()

## 그룹핑

In [ ]:
gdf = df.groupby(['약식명', '자료생성년월'])[['가입자수', '신규취득자수', '상실가입자수', '당월고지금액']].sum().reset_index()

In [ ]:
gdf

In [ ]:
gdf.columns = ["약식명", "기준일자", "직원수", "신규취득", "자격상실", "당월고지금액"]

In [ ]:
gdf["평균소득월액"] = gdf[["당월고지금액","직원수"]].astype('str').apply(get_평균소득월액, axis=1)

In [ ]:
gdf.info()

In [ ]:
gdf.head()

In [ ]:
요약통계 = gdf.groupby(['약식명'])[['직원수', '신규취득', '자격상실', '평균소득월액']].mean().round().reset_index()
요약통계

In [ ]:
gdf['기준일자'].unique()

In [ ]:
import pickle
with open ("pickle_df2.pickle", 'wb') as pickle_filename:
    pickle.dump(gdf, pickle_filename)

In [ ]:
import plotly.express as px
df = px.data.gapminder().query("continent == 'Oceania'")
df